## 求导和梯度

In [262]:
import numpy as np

In [263]:
# 函数fx在x点上的导数，这里用中心差分
def numerical_diff(f, x):
    # 偏差不能太小 计算机由于精度原因会把太小的值当作0
    delta_x = 1e-4 
    # 由于delta_x不能无限小，因此用中心差分来减少误差
    # dfx/dx = (f(x + delta) - f (x - delta)) / 2h
    return (f(x + delta_x) - f(x - delta_x)) / (2 * delta_x)


f = lambda x: x ** 2 + 3 * x + 100
numerical_diff(f, 10)


23.000000000052978

标量y对向量求导,即为该向量的梯度， 应该需要转置，即行向量的梯度为列向量

但是书上没有转制，大概后续乘法会交换顺序把。

这里按照书上来，先不转置

In [264]:
# x是向量
def numerical_gradient(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    
    # 遍历x，分别对每一个参数求导，
    # 实际上就是对当前的x[i]进行中心分差计算，其他元素不变
    for i in range(x.size):
        temp_x_i = x[i]
        x[i] = temp_x_i + delta_x
        fx1 = f(x)
        x[i] = temp_x_i - delta_x
        fx2 = f(x)
        grad[i] = (fx1 - fx2) / (2 * delta_x)
        x[i] = temp_x_i
    
    return grad

# 测试梯度函数
f = lambda a: a[0] ** 2 + 2 *a[0] * a[1] + a[2] ** 2
numerical_gradient(f, np.array([3.0, 4.0, 5.0]))


array([14.,  6., 10.])

接下来可以写出梯度下降法的函数了:x = x - lr * grad_x  
x是向量，其中lr是学习率，grad_x是x的梯度  
分解开就是:
    x[0] = x[0] - lr * grad_x[0]    
    x[1] = x[1] - lr * grad_x[1]
    ... 
    x[n] = x[n] - lr * grad_x[n]  
重复一定次数即可，求出fx的极小值/最小值了

In [265]:
# 梯度下降求函数的fx极小值的x坐标，lr系数，学习率 step重复次数
def gradient_descent(f, init_x, lr = 0.01, step=100):
    x = init_x
    for i in range(step):
        grad = numerical_gradient(f, x)
        x = x - lr * grad
    return x

# 测试下梯度下降
fx = lambda x: x[0] ** 2 + x[1] ** 2
init_x = np.array([33.0, 20.0])
gradient_descent(fx, init_x, lr=0.1)

array([6.72221872e-09, 4.07407195e-09])

定义一个简单的单层网络类，权重w是一个2 * 3的矩阵，偏置先不设置，激活函数是sofmax

可以计算出这个网络的损失函数

In [369]:
# 激活函数
def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 损失函数 交叉熵误差
def cross_entropy_error(y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

class SimpleNet:
    
    # 初始化网络    
    def __init__(self):
        super().__init__()
        self.w = np.random.randn(2, 3)

    # 推理一次 即forward    
    def predict(self, x):
        y = np.dot(x, self.w)
        y = softmax(y)
        return y

    # 网络的损失
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
        

In [372]:
# 简单测试下网络
net = SimpleNet()
print(net.w)
x = np.random.randn(1, 2)
t = np.array([0.5, 0.6, 0.7])
d = net.loss(x, t)
print(d)

[[ 0.79417551  1.69591862 -0.02072039]
 [ 0.82302452  0.50197766  1.4669881 ]]
1.9870735326459652
